# INST326 — Week 8 Exercises: Inheritance & Polymorphism (Library Management)

**Focus (Week 8 only):** subclassing, method overriding, `super()`, polymorphism via common method names, and composition vs. inheritance decisions in a small codebase.

**Out of scope (Week 9+):** abstract base classes, interfaces/protocols, multiple inheritance/mixins, advanced design patterns, decorators beyond basics, context managers beyond prior weeks, dependency injection, property descriptors beyond simple use.

> Context: Use the Library Management domain—books, members, loans, fines—to complete the tasks. Stick to basic single inheritance and straightforward overrides.


### Starter Scaffold (Week‑8‑safe)

Below is minimal starter code from prior weeks, extended slightly for Week 8. Feel free to modify it for the exercises. Avoid Week 9+ topics.


In [1]:
from __future__ import annotations
from dataclasses import dataclass
from datetime import datetime, timedelta
from typing import Dict, Optional, List

# --- Exceptions from Week 7 (basic) ---
class LibraryError(Exception): ...
class DuplicateBookError(LibraryError): ...
class OverdueLoanError(LibraryError): ...

# --- Base domain classes (no ABCs) ---
@dataclass
class Book:
    isbn: str
    title: str
    copies: int = 1

    def __str__(self) -> str:
        return f"<Book isbn={self.isbn} title={self.title}>"

    def loan_period_days(self) -> int:
        """Default loan period for a generic book."""
        return 14

    def describe(self) -> str:
        return f"Book<{self.isbn}>: {self.title} (copies={self.copies})"
    
    def __eq__(self, other) -> bool:
        if isinstance(other, Book):
            return self.isbn == other.isbn
        return False
    
    def checkout_message(self) -> str:
        return "Enjoy your book!"
    
    def can_checkout(self, stock: int) -> bool:
        return stock > 0


@dataclass
class Member:
    member_id: str
    email: str

    def max_concurrent_loans(self) -> int:
        return 5

    def describe(self) -> str:
        return f"Member<{self.member_id}>"

@dataclass
class Loan:
    isbn: str
    member_id: str
    due_date: datetime
    returned: bool = False

    def mark_returned(self) -> None:
        self.returned = True

class Catalog:
    def __init__(self):
        self._books: Dict[str, Book] = {}

    def add_book(self, book: Book) -> None:
        if book.isbn in self._books:
            raise DuplicateBookError(f"ISBN already exists: {book.isbn}")
        if book.copies < 0:
            raise ValueError("copies must be non-negative")
        self._books[book.isbn] = book

    def get_book(self, isbn: str) -> Optional[Book]:
        return self._books.get(isbn)

class LoanDesk:
    """Very small service; deliberately simple for Week 8 examples."""
    def __init__(self, catalog: Catalog, notifier: Optional[notifier] = None):
        self.catalog = catalog
        self.loans: List[Loan] = []
        self.notifier = notifier

    def checkout(self, member: Member, book: Book) -> Loan:
        # naive stock check
        active_loans = sum(1 for l in self.loans if l.member_id == member.member_id and not l.returned)
        if active_loans >= member.max_concurrent_loans():
            raise LibraryError("loan limit reached")
        if book.copies <= 0:
            raise LibraryError("no available copies")
        book.copies -= 1
        due = datetime.now() + timedelta(days=book.loan_period_days())
        loan = Loan(isbn=book.isbn, member_id=member.member_id, due_date=due)
        self.loans.append(loan)
        return loan
        if self.notifier:
            self.notifier.notify(member, book.checkout_message())
        if not book.can_checkout(book.copies):
            raise LibraryError("no available copies for this book")
    def checkin(self, loan: Loan) -> None:
        if not loan.returned:
            b = self.catalog.get_book(loan.isbn)
            if b:
                b.copies += 1
            loan.mark_returned()


## 1) Subclass a Book type

Create a subclass `PrintedBook(Book)` that overrides `loan_period_days()` to 21 days and `describe()` to include the word 'Printed'.

In [2]:
# Your code here
class PrintedBook(Book):
    def __str__(self) -> str:
        return f"<PrintedBook isbn={self.isbn} title={self.title}>"
    
    def loan_period_days(self) -> int:
        return 21

    def describe(self) -> str:
        return f"Printed {super().describe()}"

    def daily_late_fee(self) -> float:
        return 0.25

    def checkout_message(self) -> str:
        return "Enjoy your printed book!"

    def __str__(self) -> str:
        return f"<PrintedBook isbn={self.isbn} title={self.title}>"
    
    def checkout_message(self) -> str:
        return "Enjoy your printed book!"

# Example:
pb = PrintedBook("111", "Intro to Python", 2)
pb.loan_period_days()  # 21
pb.describe()          # includes 'Printed'

'Printed Book<111>: Intro to Python (copies=2)'

## 2) Another Book subtype

Create `EBook(Book)` that has an extra attribute `file_size_mb: float` (add to `__init__`), uses a 14‑day loan, and overrides `describe()` to show the size.

In [3]:
# Your code here
class EBook(Book):
    def __init__(self, isbn: str, title: str, copies: int = 1, file_size_mb: float = 0.0):
        super().__init__(isbn, title, copies)
        self.file_size_mb = file_size_mb

    def describe(self) -> str:
        return f"EBook<{self.isbn}>: {self.title} ({self.file_size_mb} MB)"

    def daily_late_fee(self) -> float:
        return 0.10

    def checkout_message(self) -> str:
        return "Enjoy your eBook!"

    def download_link(self) -> str:
        return f"https://library.example.com/download/{self.isbn}"

    def can_checkout(self, stock: int) -> bool:
        return stock >= 0
    
    def checkout_message(self) -> str:
        return "Enjoy your eBook!"

## 3) Override with super()

Create `AudioBook(Book)` with extra field `duration_min: int`. Override `describe()` to start with `super().describe()` and append `duration_min`.

In [4]:
# Your code here
class AudioBook(Book):
    def __init__(self, isbn: str, title: str, copies: int = 1, duration_min: int = 0):
        super().__init__(isbn, title, copies)
        self.duration_min = duration_min

    def describe(self) -> str:
        return f"{super().describe()} (duration={self.duration_min} min)"

    def daily_late_fee(self) -> float:
        return 0.15

    def checkout_message(self) -> str:
        return "Enjoy your audiobook!"


## 4) Non‑circulating subclass

Create `ReferenceBook(Book)` that **cannot** be checked out. Override `loan_period_days()` to return `0`. In `LoanDesk.checkout`, demonstrate a guard that raises `LibraryError('non-circulating')` if the period is 0.

In [5]:
# Your code here
class ReferenceBook(Book):
    def loan_period_days(self) -> int:
        return 0

    def daily_late_fee(self) -> float:
        return 0.0

    def checkout_message(self) -> str:
        return "Reference books cannot be checked out."

# Update/extend LoanDesk.checkout minimally to guard non‑circulating

## 5) Member specialization

Create `Student(Member)` and `Staff(Member)`. Students can have 5 concurrent loans; staff 10. Override `max_concurrent_loans()` accordingly.

In [6]:
# Your code here
class Student(Member):
    def max_concurrent_loans(self) -> int:
        return 5
class Staff(Member):
    def max_concurrent_loans(self) -> int:
        return 10

## 6) Polymorphic fine calculation

Write a function `late_fee(book: Book, days_late: int) -> float` that uses polymorphic behavior:
- PrintedBook: $0.25/day
- EBook: $0.10/day
- AudioBook: $0.15/day
- Fallback (Book): $0.20/day
Use `isinstance` checks only; do not modify the classes for this one.

In [7]:
# Your code here
def late_fee(book: Book, days_late: int) -> float:
    if isinstance(book, PrintedBook):
        rate = 0.25
    elif isinstance(book, EBook):
        rate = 0.10
    elif isinstance(book, AudioBook):
        rate = 0.15
    else:
        rate = 0.20
    return rate * days_late

## 7) Polymorphism without isinstance

Refactor your approach so **each subclass** implements `daily_late_fee()` and `late_fee(days_late)` (calling `daily_late_fee()`), then write a single function `compute_fee(book: Book, days_late: int)` that calls `book.late_fee(days_late)` without type checks.

In [8]:
# Your code here
# Add methods to Book and subclasses; avoid breaking earlier exercises if possible.
def compute_fee(book: Book, days_late: int) -> float:
    return book.late_fee(days_late)

## 8) Overriding __str__

Override `__str__` in `Book` to return `<Book isbn=... title=...>`. Override it in one subclass to include subtype info, e.g., `<PrintedBook isbn=...>`.

In [9]:
# Your code here
def __str__(self) -> str:
    return f"<Book isbn={self.isbn} title={self.title}>"
# Override in Book and one subclass

## 9) Composition vs. inheritance

Create a small `Notifier` class with method `notify(member: Member, message: str)`. Demonstrate **composition** by adding a `notifier` attribute to `LoanDesk` and using it during checkout to acknowledge a loan. Keep `Notifier` very simple (e.g., print or collect messages).

In [10]:
# Your code here
class Notifier:
    class Notifier:
        def notify(self, member: Member, message: str) -> None:
            print(f"Notification to {member.member_id}: {message}")

# Integrate into LoanDesk via composition (not inheritance)

## 10) Enforcing limits polymorphically

Modify `LoanDesk.checkout` to check a member's current active loans (for that member_id) and compare to `member.max_concurrent_loans()` before allowing checkout. Demonstrate with a `Student` hitting the limit and a `Staff` not hitting it.

In [11]:
# Your code here
# Update LoanDesk.checkout and show a brief demo

## 11) Subclass‑specific behavior

Add a method `download_link()` to `EBook` returning a fake URL string using the ISBN. Do not add this to `Book` or other subclasses. Show a short snippet where you use duck typing safely by checking `hasattr` before calling.

In [12]:
# Your code here
# Add method to EBook and a usage demo with duck typing

## 12) Polymorphic loan period by member type

Some libraries extend loan periods for `Staff`. Implement `effective_loan_period(book: Book, member: Member) -> int`:
- start from `book.loan_period_days()`
- if `isinstance(member, Staff)`, add +7 days
Return the resulting days.

In [13]:
# Your code here
def effective_loan_period(book: Book, member: Member) -> int:
    period = book.loan_period_days()
    if isinstance(member, Staff):
        period += 7
    return period

## 13) Override equality semantics (dataclass)

For `Book`, override `__eq__` so that books are considered equal iff ISBNs match (ignore title/copies). Write quick tests comparing a `PrintedBook` and `EBook` with the same ISBN—they should be equal by ISBN.

In [14]:
# Your code here
def __eq__(self, other) -> bool:
    if isinstance(other, Book):
        return self.isbn == other.isbn
    return False
# Example:
pb1 = PrintedBook("100", "Python", 1)
eb1 = EBook("100", "Python eBook", 1, file_size_mb=2.0)
assert pb1 == eb1  # True, same ISBN
# Override __eq__ on Book carefully; demonstrate with examples

## 14) Draft a small class hierarchy diagram (markdown)

In **markdown**, sketch a tiny hierarchy diagram for `Book <- PrintedBook | EBook | AudioBook | ReferenceBook` and `Member <- Student | Staff`. No code—just a clear diagram using text/ASCII.

In [15]:

"""
Book
 ├── PrintedBook
 ├── EBook
 ├── AudioBook
 └── ReferenceBook

Member
 ├── Student
 └── Staff
"""

'\nBook\n ├── PrintedBook\n ├── EBook\n ├── AudioBook\n └── ReferenceBook\n\nMember\n ├── Student\n └── Staff\n'

## 15) Replace conditional with polymorphism

Currently, `LoanDesk.checkout` always uses `book.loan_period_days()`. Add an overridable method `checkout_message()` to `Book` and override it in at least two subclasses to customize the user‑facing message returned by `LoanDesk.checkout` (e.g., 'Enjoy your audiobook!'). Show the different messages without `if/elif` chains.

In [16]:
# Your code here
# Add checkout_message to Book and override in subclasses; update LoanDesk.checkout to use it

## 16) Subclass‑specific stock policy

Override `LoanDesk.checkout` to deny checkout of `EBook` if `copies < 0` (simulate licensing depletion), but allow `PrintedBook` as long as `copies > 0`. Implement this by relying on each subclass's own `can_checkout(stock: int) -> bool` method. Default in `Book` should be `stock > 0`.

In [17]:
# Your code here
# Add can_checkout to Book and subclasses; update LoanDesk.checkout accordingly

## 17) Sorting polymorphically

Create a list mixing `PrintedBook`, `EBook`, and `AudioBook`. Implement a function `sort_books_for_display(books: list[Book]) -> list[Book]` that sorts by this precedence: Printed first, then EBook, then AudioBook; ties broken by title. Use a key function that relies on `isinstance` or a small polymorphic `display_rank()` method.

In [18]:
# Your code here
def sort_books_for_display(books: list[Book]) -> list[Book]:
    def display_rank(b: Book):
        if isinstance(b, PrintedBook):
            return 1
        if isinstance(b, EBook):
            return 2
        if isinstance(b, AudioBook):
            return 3
        return 4
    return sorted(books, key=lambda b: (display_rank(b), b.title))


## 18) Minimal polymorphic report

Write `summarize_books(books: list[Book]) -> list[str]` that returns `describe()` for each. Show that the correct overridden `describe()` is used without `if/elif`.

In [19]:
# Your code here
def summarize_books(books: list[Book]) -> list[str]:
    return [b.describe() for b in books]

## 19) Unit test: overriding works

Using `unittest`, add a small test class that checks `loan_period_days()` for `PrintedBook` (21) and `ReferenceBook` (0), and that `__str__` includes the subclass name for one subtype.

In [20]:
# Your code here
import unittest

class TestWeek8Inheritance(unittest.TestCase):
    def test_loan_periods_and_str(self):
        pb = PrintedBook("001", "Python", 1)
        rb = ReferenceBook("002", "Encyclopedia", 1)
        self.assertEqual(pb.loan_period_days(), 21)
        self.assertEqual(rb.loan_period_days(), 0)
        self.assertIn("PrintedBook", str(pb))

# # To run inside notebook:
# # unittest.main(argv=['-v'], exit=False)

## 20) Polymorphic fee scenario (end‑to‑end)

Create a short demo that:
- Builds a `Catalog` and `LoanDesk` (with `Notifier` if you implemented it)
- Adds one book of each subtype
- Checks each out to a `Student`
- Simulates `days_late` values and prints fees using your polymorphic fee API
Show that different subtypes yield different fees without `if/elif` at the call site.

In [21]:
# Your code here
# End-to-end demo using your polymorphic methods
if __name__ == "__main__":
    catalog = Catalog()
    notifier = Notifier()
    desk = LoanDesk(catalog, notifier)

    pb = PrintedBook("001", "Python 101", 2)
    eb = EBook("002", "Data Science", 1, file_size_mb=5.0)
    ab = AudioBook("003", "Learn Python Audio", 1, duration_min=120)
    catalog.add_book(pb)
    catalog.add_book(eb)
    catalog.add_book(ab)

    student = Student("s01", "s01@example.com")
    staff = Staff("st01", "st01@example.com")

    # Checkout books
    loans = [desk.checkout(student, b) for b in [pb, eb, ab]]

    # Simulate days_late and print fees
    for loan, b in zip(loans, [pb, eb, ab]):
        print(f"{b.title} late fee for 3 days: ${compute_fee(b, 3):.2f}")

AttributeError: 'PrintedBook' object has no attribute 'late_fee'

## Python skills you'll need (Weeks 1–8)

- **Core syntax & data types:** variables, strings, numbers, booleans
- **Collections:** lists, dicts (basic use), simple list/dict comprehensions
- **Control flow:** `if/elif/else`, `for`, `while`
- **Functions & modules:** defining functions, parameters, returns, imports
- **File I/O & JSON (basic):** open/read/write, simple JSON usage
- **Classes & objects (Weeks 4–6):** defining classes, attributes, methods, `__init__`, `__str__`
- **Encapsulation basics:** simple validation; naming conventions for "private" attributes
- **Methods:** instance/class/static methods (as introduced up to Week 6)
- **Error handling & testing (Week 7):** `try/except/else/finally`, custom exceptions, basic `unittest`
- **Week 8 focus:** **single inheritance**, **method overriding**, **`super()`**, **polymorphism via common methods**, and **composition vs. inheritance** decisions
- **Standard library familiarity:** `datetime`, `timedelta`, built‑in exceptions
